In [ ]:
"""

當用戶加入群組，我們會收到 MemberJoinedEvent

此時我們可以透過 groupId 與 userId 向 LINE 官方取得用戶個資

不論此用戶是否有關注我方官方帳號

profile = line_bot_api.get_group_member_profile(group_id, user_id)


用戶個資包含
    profile.display_name
    profile.user_id
    profile.picture_url


"""

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""

人類 被加入群組時觸發

機器人 向新加入的人類問好

"""

from linebot.models import (
    MemberJoinedEvent, TextSendMessage, ImageSendMessage
)

import json

@handler.add(MemberJoinedEvent)
def handle_member_join(event):
    print(event)
    
    group_id = event.source.group_id
    joined_members_list = event.joined.members
    
    members_name_list = []
    for member in joined_members_list:
        
        # 從 members 串列 中一個一個取出 member
        # 並利用取出的 member user id 與群組 id，向 LINE 官方取得用戶個資
        user_id = member.user_id
        profile = line_bot_api.get_group_member_profile(group_id, user_id)
        
        # 將用戶個資存檔
        with open('user_in_the_group.json', 'a', encoding='utf-8') as f:
            user_profile_dict = profile.__dict__
            user_profile_dict['group_id'] = group_id
            f.write(json.dumps(user_profile_dict))
            f.write('\n')
        
        # 準備用戶名字串列，等等用來打招呼
        members_name_list.append(profile.display_name)

    members_name_str = ", ".join(members_name_list)
    
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=members_name_str + " 你(們)好\n我是這個群組唯一的機器人，請多多指教！")
    )

In [ ]:

'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')